In [3]:
import yfinance as yf

def evaluate_etf_fundamentals(info):
    """
    ETF fundamentális mutatók alapján történő értékelés:
      - Expense Ratio: Az alacsonyabb érték jobb.
          * Ideális: < 0.002 (0.2%)
          * Határ: < 0.005 (0.5%)
      - Dividend Yield: A magasabb osztalékhozam előnyös.
          * Ideális: > 0.03 (3%)
          * Határ: > 0.01 (1%)
      - Likviditás: Az átlagos napi forgalom (10 napos) alapján.
          * Ideális: > 1,000,000 részvény
          * Határ: > 500,000 részvény
      - Kezelt vagyon (Total Assets): Ha elérhető, a nagyobb méret stabilitást jelez.
          * Ideális: > 10 milliárd USD
          * Határ: > 5 milliárd USD
    """
    score = 0
    print("\n--- ETF Fundamentális mutatók ---")
    
    # Expense Ratio
    exp_ratio = info.get("expenseRatio")
    if exp_ratio is not None:
        print("Expense Ratio:", exp_ratio)
        if exp_ratio < 0.002:
            score += 2
        elif exp_ratio < 0.005:
            score += 1
    else:
        print("Expense Ratio adat nem elérhető.")
    
    # Dividend Yield
    dividend_yield = info.get("dividendYield")
    if dividend_yield is not None:
        print("Dividend Yield:", dividend_yield)
        if dividend_yield > 0.03:
            score += 2
        elif dividend_yield > 0.01:
            score += 1
    else:
        print("Dividend Yield adat nem elérhető.")
    
    # Likviditás (10 napos átlagos napi forgalom)
    volume = info.get("averageDailyVolume10Day")
    if volume is not None:
        print("Average Daily Volume (10 nap):", volume)
        if volume > 1_000_000:
            score += 2
        elif volume > 500_000:
            score += 1
    else:
        print("Average Daily Volume adat nem elérhető.")
    
    # Kezelt vagyon (Total Assets)
    total_assets = info.get("totalAssets")
    if total_assets is not None:
        print("Total Assets:", total_assets)
        try:
            total_assets = float(total_assets)
        except Exception as e:
            total_assets = None
        if total_assets is not None:
            if total_assets > 10e9:
                score += 2
            elif total_assets > 5e9:
                score += 1
    else:
        print("Total Assets adat nem elérhető.")
    
    return score

def evaluate_etf_technical(ticker):
    """
    Egyszerű technikai elemzés ETF-ekre:
      - Lekéri az elmúlt 1 év (kb. 250 kereskedési nap) adatait,
      - Számolja az 50 napos és 200 napos mozgóátlagokat,
      - Ha az aktuális ár meghaladja a 50 napos és/vagy a 200 napos mozgóátlagot, plusz pontokat ad.
    """
    hist = ticker.history(period="1y")
    if hist.empty:
        print("Nincs elérhető történelmi adat.")
        return 0
    current_price = hist['Close'].iloc[-1]
    ma50 = hist['Close'].rolling(window=50).mean().iloc[-1]
    ma200 = hist['Close'].rolling(window=200).mean().iloc[-1]
    
    tech_score = 0
    print("\n--- ETF Technikai elemzés ---")
    print("Aktuális ár:", current_price)
    print("50 napos mozgóátlag:", ma50)
    print("200 napos mozgóátlag:", ma200)
    
    if current_price > ma50:
        tech_score += 1
    if current_price > ma200:
        tech_score += 1
    
    return tech_score

def evaluate_etf_portfolio(info):
    """
    ETF portfólióösszetétel és diverzifikáció elemzése:
      - Ha elérhető a "sectorWeightings" mező (lista formában), kiértékeljük,
        hogy mennyire diverzifikált az ETF.
      - Példa: ha egy szektor súlya meghaladja az 50%-ot, az koncentrált portfóliót jelez,
        így negatív pontot adunk.
      - Ha a legnagyobb szektor súlya alacsony, az jobb diverzifikációt jelent, és plusz pont jár érte.
    """
    portfolio_score = 0
    print("\n--- ETF Portfólióösszetétel elemzése ---")
    sector_weightings = info.get("sectorWeightings")
    if sector_weightings is not None and isinstance(sector_weightings, list):
        # Az elvárt formátum: lista, például:
        # [{"sector": "Technology", "weight": 30.0}, {"sector": "Financials", "weight": 20.0}, ...]
        max_weight = 0
        for sector in sector_weightings:
            weight = sector.get("weight", 0)
            print(f"{sector.get('sector', 'Ismeretlen')}: {weight}%")
            if weight > max_weight:
                max_weight = weight
        print("Legnagyobb szektor súlya:", max_weight, "%")
        if max_weight < 30:
            portfolio_score += 2
        elif max_weight < 50:
            portfolio_score += 1
        else:
            portfolio_score -= 1  # Túlzott koncentráció
    else:
        print("Portfólió összetétel adat nem elérhető.")
    return portfolio_score

def evaluate_etf_complex(info):
    """
    További komplex elemzés ETF-ek esetén:
      - Beta: Ha elérhető, értékeljük az ETF volatilitását.
          * Ha beta < 1, akkor alacsony volatilitás (pozitív pont).
          * Ha beta > 1.5, negatív pontot adunk.
      - Tracking Error: Ha elérhető, az indexkövetés hatékonyságát mérjük.
          * Alacsony tracking error (< 0.05) jó.
    """
    complex_score = 0
    print("\n--- ETF Komplex elemzés ---")
    beta = info.get("beta")
    if beta is not None:
        print("Beta:", beta)
        try:
            beta = float(beta)
            if beta < 1:
                complex_score += 2
            elif beta < 1.5:
                complex_score += 1
            else:
                complex_score -= 1
        except Exception as e:
            print("Beta érték átalakítása nem sikerült.")
    else:
        print("Beta adat nem elérhető.")
    
    tracking_error = info.get("trackingError")
    if tracking_error is not None:
        print("Tracking Error:", tracking_error)
        try:
            tracking_error = float(tracking_error)
            if tracking_error < 0.05:
                complex_score += 2
            elif tracking_error < 0.1:
                complex_score += 1
            else:
                complex_score -= 1
        except Exception as e:
            print("Tracking Error átalakítása nem sikerült.")
    else:
        print("Tracking Error adat nem elérhető.")
    
    return complex_score

def overall_etf_recommendation(total_score):
    """
    Az összesített pontszám alapján ad egy egyszerű befektetési ajánlást:
      - 10 vagy több pont: Nagyon ajánlott ETF befektetés.
      - 6–9 pont: Közepesen ajánlott, további kutatás javasolt.
      - Kevesebb, mint 6 pont: Nem ajánlott ETF befektetés.
    """
    print("\nÖsszpontszám:", total_score)
    if total_score >= 10:
        return "Nagyon ajánlott ETF befektetés."
    elif total_score >= 6:
        return "Közepesen ajánlott ETF befektetés, de további kutatás javasolt."
    else:
        return "Nem ajánlott ETF befektetés."

def main():
    ticker_code = input("Adja meg az ETF ticker kódját (pl. SPY, QQQ, stb.): ").strip().upper()
    ticker = yf.Ticker(ticker_code)
    
    try:
        info = ticker.info
    except Exception as e:
        print("Hiba történt az adatok lekérése során:", e)
        return
    
    if not info:
        print("Nem sikerült lekérni az adatokat a megadott ticker kódhoz.")
        return
    
    name = info.get("shortName") or info.get("longName")
    if name:
        print("\nETF neve:", name)
    else:
        print("\nETF neve nem elérhető.")
    
    # ETF Fundamentális elemzés
    fund_score = evaluate_etf_fundamentals(info)
    # ETF Technikai elemzés
    tech_score = evaluate_etf_technical(ticker)
    # ETF Portfólió összetétel elemzése
    portfolio_score = evaluate_etf_portfolio(info)
    # További komplex elemzés (Beta, Tracking Error)
    complex_score = evaluate_etf_complex(info)
    
    total_score = fund_score + tech_score + portfolio_score + complex_score
    recommendation = overall_etf_recommendation(total_score)
    print("\nBefektetési ajánlás:", recommendation)

if __name__ == "__main__":
    main()



ETF neve: Invesco QQQ Trust, Series 1

--- ETF Fundamentális mutatók ---
Expense Ratio adat nem elérhető.
Dividend Yield adat nem elérhető.
Average Daily Volume (10 nap): 33883520
Total Assets: 318905516032

--- ETF Technikai elemzés ---
Aktuális ár: 522.2899780273438
50 napos mozgóátlag: 517.8209661865235
200 napos mozgóátlag: 481.44010116577147

--- ETF Portfólióösszetétel elemzése ---
Portfólió összetétel adat nem elérhető.

--- ETF Komplex elemzés ---
Beta adat nem elérhető.
Tracking Error adat nem elérhető.

Összpontszám: 6

Befektetési ajánlás: Közepesen ajánlott ETF befektetés, de további kutatás javasolt.
